# PRML Assignment 4

@uthor: Darshil Patel(202011034)

In [121]:
#useful librarys for this assigment
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [122]:
def load_iris_data():
  data = pd.read_csv(filepath_or_buffer='https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header=None,sep=',')
  data.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'flower_class']
  data.dropna(how="all", inplace=True) # drops the empty line at file-end
  return data

# data split function
def my_train_test_split(data_frame,test_size):
  grouped = data_frame.groupby(data_frame.flower_class)
  setosa = grouped.get_group(0)
  versicolor = grouped.get_group(1)
  virginica = grouped.get_group(2)

  # fetures and labels
  x_setosa = setosa.iloc[:,:-1]
  y_setosa = setosa.iloc[:,-1]
  x_versicolor = versicolor.iloc[:,:-1]
  y_versicolor = versicolor.iloc[:,-1]
  x_virginica = virginica.iloc[:,:-1]
  y_virginica = virginica.iloc[:,-1]

  # randomely spliting
  x_train_setosa, x_test_setosa, y_train_setosa, y_test_setosa  = train_test_split(x_setosa, y_setosa,test_size=test_size)
  x_train_versicolor, x_test_versicolor, y_train_versicolor, y_test_versicolor= train_test_split(x_versicolor, y_versicolor, test_size=test_size)
  x_train_virginica, x_test_virginica,  y_train_virginica, y_test_virginica,= train_test_split(x_virginica, y_virginica, test_size=test_size)

  # train data and label
  x_train = pd.concat([x_train_setosa , x_train_versicolor, x_train_virginica])
  y_train = list(pd.concat([y_train_setosa, y_train_versicolor, y_train_virginica]))
  
  # test data and label
  x_test = pd.concat([x_test_setosa , x_test_versicolor, x_test_virginica])
  y_test = list(pd.concat([y_test_setosa, y_test_versicolor, y_test_virginica]))

  return x_train,x_test,y_train,y_test

# get accuracy from test label and predicted label
def accuracy(y_test,y_predict):
  error = 0
  for x,y in zip(y_test,y_predict):
    if x != y:
      error += 1
  ac = ((len(y_test)-error)/len(y_test))*100
  return ac

In [123]:
# Naive Bayes classifer class
class NaiveBayes():
  # initialized parameters
  def __init__(self):
    self.parameters = dict()
    self.class_probability = dict()

  # fit method to train data
  def fit(self,data,label):

    # total number of data
    n = len(data)

    # seperate data by class labels
    seperated_data = dict()
    for i in range(n):
      sample = data[i]
      class_label = label[i]
      if (class_label not in seperated_data):
        seperated_data[class_label] = list()
      seperated_data[class_label].append(sample)

    # calculate mean and standard deviation of each feature of each class
    self.class_probability = dict()
    self.parameters = dict()
    for class_label,samples in seperated_data.items():
      self.class_probability[class_label] = len(samples)/n
      self.parameters[class_label] = [[np.mean(column), np.std(column)] for column in zip(*samples)]

  # probability of sample for considering class as gaussian distribution 
  def P(self,x,mean,stdev):
    exponent = np.exp(-((x-mean)**2 / (2 * stdev**2 )))
    return (1 / (np.sqrt(2 * np.pi) * stdev)) * exponent

  # P(x/y) where x is sample and y is class(label) 
  def P_of_x_given_y(self,x,y):
    ans = 1
    for value,par in zip(x,self.parameters[y]):
      ans = ans*self.P(value,par[0],par[1])
    return ans
    
  # P(x) where x is sampe 
  def P_of_x(self,x):
    ans = 0
    for class_label in self.class_probability:
      ans += self.P_of_x_given_y(x,class_label)*self.class_probability[class_label]
    return ans

  # P(Y/X) where x is sample and y is class(label) 
  def P_of_y_given_x(self,y,x):
    return (self.P_of_x_given_y(x,y)*self.class_probability[y])/self.P_of_x(x)
  
  # predect the labels of test data 
  def predict(self,data):
    y_predict = []
    for i in range(len(data)):
      classwise_probability = dict()
      predicted_class = None
      predicted_class_probability = -1

      # calculate class wise probability 
      for class_label in self.class_probability:
        classwise_probability[class_label] = self.P_of_y_given_x(class_label,data[i])
        
        # select class with maximum probability
        if classwise_probability[class_label] >= predicted_class_probability:
          predicted_class = class_label
          predicted_class_probability = classwise_probability[class_label]

      # predict labels
      y_predict.append(predicted_class)
    return y_predict

    

In [124]:
# load data
data = load_iris_data()

# encode labels 
label_encoder = preprocessing.LabelEncoder()
data['flower_class']= label_encoder.fit_transform(data['flower_class'])
data.head()

,sepal_len,sepal_wid,petal_len,petal_wid,flower_class
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [125]:
sum_ac = 0
for _ in range(10):
  print("-------------------------Iteration",_+1,"-------------------------")
  # split dataset 
  x_train,x_test,y_train,y_test = my_train_test_split(data,test_size=0.2)

  # convert to numpy metrix
  x_train = x_train.to_numpy()
  x_test = x_test.to_numpy()

  # create model
  naive_bayes = NaiveBayes()

  # train model
  naive_bayes.fit(x_train,y_train)

  # test model
  y_predict = naive_bayes.predict(x_test)

  # model's accuracy
  ac = accuracy(y_test,y_predict)
  print("original classes:-\n",list(y_test))
  print("predicted classes:-\n",y_predict)
  print("accuracy :-",ac,'%')
  sum_ac += ac

print('--------------------------------')

# model's average accuracy 
ans = sum_ac/10
print("model's accuracy :-",ans)

-------------------------Iteration 1 -------------------------
original classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
predicted classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
accuracy :- 96.66666666666667 %
-------------------------Iteration 2 -------------------------
original classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
predicted classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2]
accuracy :- 96.66666666666667 %
-------------------------Iteration 3 -------------------------
original classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
predicted classes:-
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
accuracy :- 93.33333333333333 %
-------------------------Iteration 4 ---------

In [130]:
# implement using sklearn module

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

x_train,x_test,y_train,y_test = my_train_test_split(data,test_size=0.2)
classifier = GaussianNB()
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

print ("Accuracy : ", accuracy_score(y_test, y_pred)*100)

Accuracy :  96.66666666666667
